In [2]:
import os, sys, glob
import logging
from datetime import datetime 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 

In [3]:
#from newspaper3k import Article

In [4]:
from twisted.internet import reactor
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
from scrapy.settings import Settings

In [5]:
class TargetObject: 
    def __init__(self, urlz, headerz, parser_xtra ):
        self.urlz = urlz
        self.headerz = headerz
        self.parser_xtra = parser_xtra
        print("Target: ", self.urlz)
        


In [6]:
class ArticlesFinder(scrapy.Spider):
    name = "articles_finder"    

    def start_requests(self):
        print('starting articles_finder')
        self.target = getattr(self, 'target', None)
        print('target = ', target.__dict__)
        if self.target.urlz:
            for url in self.target.urlz:
                yield scrapy.Request(url=url, 
                                     callback=self.parse,
                                     headers={'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"},
                                     dont_filter=True,
                                    meta={"dont_merge_cookies": True, 'position':0})
    
    def parse(self, response): 
        print("\n****** starting parser - ") 
        page = response.url.split("/")[-2]
        result = response   
        if self.target.parser_xtra:
            yield self.target.parser_xtra( result )
        else:
            yield result.body
        

In [6]:
# urlz = ['https://www.thelancet.com/coronavirus', ]
##TODO: item pipelines 
name = "articles_finder" 

logger = {}
# logger = {'LOG_LEVEL':logging.WARNING,}

def crawl(target, fname, logger=logger):
    config = get_project_settings() 
    config.update({'target': target,
                  'FEED_FORMAT':'csv',
                  'FEED_URI':fname, 
                  #**logger
                  })

    proc = CrawlerProcess( config )
    # proc.crawl( ArticlesFinder )
    #proc.crawl('quotes', 'dummyinput', **custom_settings_spider)
    proc.crawl(ArticlesFinder, 'dummyinput', **config )
    proc.start()
    #reactor.stop()
    print('FIN: ', fname)

urlz = ['https://en.wikiquote.org/wiki/Marilyn_Manson', ]
headerz = {}

def parser_p(x):
    for item in x.css('div.mw-parser-output > ul > li'):
        return {'item':item.extract()}

target = TargetObject(urlz, headerz, parser_p)

# crawl(target, 'test.csv')

Target:  ['https://en.wikiquote.org/wiki/Marilyn_Manson']


# 2. Let's Quick and Dirty Find


In [7]:
from multiprocessing import Process, Queue
from scrapy import crawler
from twisted.internet import reactor
from scrapy.utils.log import configure_logging

configure_logging({'LOG_FORMAT': '%(levelname)s: %(message)s'})
    
def run_spider(spider, config):
    def f(q):
        try:
            runner = crawler.CrawlerRunner(config)
            deferred = runner.crawl(spider, **config )
            deferred.addBoth( lambda _: reactor.stop() )
            reactor.run() 
            q.put(None)
            print( "RAN ")
        except Exception as e:
            q.put(e)
    q = Queue()
    p = Process(target=f, args=(q,) )
    p.start()
    result = q.get()
    p.join() 
    if result is not None:
        raise result

In [8]:
name = "articles_finder" 
#logger = {'LOG_LEVEL':logging.WARNING,}

def crawl_articles(target, fname, logger=logger):
    config = get_project_settings() 
    config.update({'target': target, 
                  'FEEDS': { fname:{ 'format': 'csv',}}, ##f"~/Repoz/school/02_ocular/{fname}", 
                  #**logger
                  })
#     print( config.keys() )
    run_spider(ArticlesFinder, config) 
    print('FIN: ', fname)


In [9]:
## 1. Elsevier
def parser_gscholar(x):
    for item in x.css('div.mw-parser-output > ul > li'):
        print( item.extract() )
        return {'item':item.extract()}

#/html/body/main/div[1]/div/div/div/div/div[2]/div[2]/div
def parser_elsevier(x): #div.search-result-items > 
    for item in x.css('article > header'): 
        res = {'title': item.css('h2').extract(),
               'text': item.css('div.search-result-text ').extract(),
               'link': item.css('ul > li > a').extract(),}
        print( res )
        return res

def parser_ieee(x):
    return x

def parser_jama(x):
    return x

def parser_nature(x):
    return x

def parser_lancet(x):
    return x


In [10]:
# 'https://scholar.google.com/scholar?' + urlencode({'hl': 'en', 'q': query})

## Url, Headerz, Parser, Fname = u, h, p, f 
headerz = {'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"}
targetz = [(['https://en.wikiquote.org/wiki/Marilyn_Manson',], {'DEFAULT_REQUEST_HEADERS': headerz}, parser_gscholar, "gscholar.csv"),
           (['https://www.elsevier.com/search-results?query=fundus%20handcrafted%20features&labels=journals',], {'DEFAULT_REQUEST_HEADERS': headerz}, parser_elsevier, "elsevier.csv"), 
          ]
for u, h, p, f in targetz:
    target = TargetObject(u, h, p)
    crawl_articles(target, f) 

Target:  ['https://en.wikiquote.org/wiki/Marilyn_Manson']


INFO: Overridden settings:
{}
2021-03-16 16:43:19 [scrapy.extensions.telnet] INFO: Telnet Password: c893391c53a0f51c
2021-03-16 16:43:19 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-03-16 16:43:19 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware'

starting articles_finder
target =  {'urlz': ['https://en.wikiquote.org/wiki/Marilyn_Manson'], 'headerz': {'DEFAULT_REQUEST_HEADERS': {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1'}}, 'parser_xtra': <function parser_gscholar at 0x7eff64effa60>}


2021-03-16 16:43:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikiquote.org/wiki/Marilyn_Manson> (referer: None)



****** starting parser - 
<li>If people really stopped and realized how much art and creative people move the world versus politics and religion, I mean it’s not even up for debate. An artist at least creates things, puts things into the world. Where as these other people are destroying things, taking things out of the world.
<ul><li>Interview on <i>The Henry Rollins Show</i></li></ul></li>


...

starting articles_finder
target =  {'urlz': ['https://www.elsevier.com/search-results?query=fundus%20handcrafted%20features&labels=journals'], 'headerz': {'DEFAULT_REQUEST_HEADERS': {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1'}}, 'parser_xtra': <function parser_elsevier at 0x7eff64effae8>}


2021-03-16 16:43:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.elsevier.com/search-results?query=fundus%20handcrafted%20features&labels=journals> (referer: None)



****** starting parser - 
{'title': ['<h2 class="search-result-title search-result-journal-title" data-v-7cc532f4><a href="https://www.journals.elsevier.com/journal-of-prosthetic-dentistry" data-v-7cc532f4>Journal of Prosthetic Dentistry</a></h2>'], 'text': [], 'link': []}


2021-03-16 16:43:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.elsevier.com/search-results?query=fundus%20handcrafted%20features&labels=journals>
{'title': ['<h2 class="search-result-title search-result-journal-title" data-v-7cc532f4><a href="https://www.journals.elsevier.com/journal-of-prosthetic-dentistry" data-v-7cc532f4>Journal of Prosthetic Dentistry</a></h2>'], 'text': [], 'link': []}
2021-03-16 16:43:24 [scrapy.core.engine] INFO: Closing spider (finished)
2021-03-16 16:43:24 [scrapy.extensions.feedexport] INFO: Stored csv feed (1 items) in: elsevier.csv
2021-03-16 16:43:24 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 355,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 65122,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'elapsed_time_seconds': 3.794108,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 3, 16, 13, 43, 2

RAN 
FIN:  elsevier.csv


# Scopus API - Elsevier

https://dev.elsevier.com/



https://kitchingroup.cheme.cmu.edu/blog/2015/04/03/Getting-data-from-the-Scopus-API/ 


PyScopus http://zhiyzuo.github.io/python-scopus/ 


In [16]:
import json

# logger = {'LOG_LEVEL':logging.INFO,}
# configure_logging({'LOG_FORMAT': '%(levelname)s: %(message)s'})
MY_API_KEY = '6f08887e7863302c0431a454c96fb54c'
headerz_api={'Accept':'application/json', 'X-ELS-APIKey': MY_API_KEY}

def parser_elsevier_api(x): #div.search-result-items > 
    for item in json.loads(x.body_as_unicode().replace("'", '"')): 
        for entry in resp.json()['search-results']['entry']: 
            res = {k:v for k, v in entry if k in ('dc:title', 'prism:coverDate', 'prism:url',
                                                    'citedby-count', 'affiliation', 
                                                    'prism:aggregationType') }
            print(res)
            return res

targetz = [(['https://en.wikiquote.org/wiki/Marilyn_Manson',], 
            {'DEFAULT_REQUEST_HEADERS': headerz, }, 
            parser_gscholar, "gscholar.csv"),
           
           (["http://api.elsevier.com/content/search/scopus?query=fundus",], 
            {'headers': headerz_api}, 
            parser_elsevier_api, "elsevier_api.csv"), 
  ]

for u, h, p, f in targetz:
    target = TargetObject(u, h, p)
    crawl_articles(target, f) 

Target:  ['https://en.wikiquote.org/wiki/Marilyn_Manson']


INFO: Overridden settings:
{}
2021-03-16 16:44:49 [scrapy.extensions.telnet] INFO: Telnet Password: deb1c6186ee95888
2021-03-16 16:44:49 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-03-16 16:44:49 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware'

starting articles_finder
target =  {'urlz': ['https://en.wikiquote.org/wiki/Marilyn_Manson'], 'headerz': {'DEFAULT_REQUEST_HEADERS': {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1'}}, 'parser_xtra': <function parser_gscholar at 0x7eff64effa60>}


2021-03-16 16:44:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikiquote.org/wiki/Marilyn_Manson> (referer: None)



****** starting parser - 
<li>If people really stopped and realized how much art and creative people move the world versus politics and religion, I mean it’s not even up for debate. An artist at least creates things, puts things into the world. Where as these other people are destroying things, taking things out of the world.
<ul><li>Interview on <i>The Henry Rollins Show</i></li></ul></li>


...

RAN 
FIN:  gscholar.csv
Target:  ['http://api.elsevier.com/content/search/scopus?query=fundus']


INFO: Overridden settings:
{}
2021-03-16 16:44:50 [scrapy.extensions.telnet] INFO: Telnet Password: 9d534b88d0de13c0
2021-03-16 16:44:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-03-16 16:44:50 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware'

starting articles_finder
target =  {'urlz': ['http://api.elsevier.com/content/search/scopus?query=fundus'], 'headerz': {'headers': {'Accept': 'application/json', 'X-ELS-APIKey': '6f08887e7863302c0431a454c96fb54c'}}, 'parser_xtra': <function parser_elsevier_api at 0x7eff648ce840>}


2021-03-16 16:44:51 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (308) to <GET https://api.elsevier.com/content/search/scopus?query=fundus> from <GET http://api.elsevier.com/content/search/scopus?query=fundus>
2021-03-16 16:44:52 [scrapy.core.engine] DEBUG: Crawled (401) <GET https://api.elsevier.com/content/search/scopus?query=fundus> (referer: None)
2021-03-16 16:44:52 [scrapy.spidermiddlewares.httperror] INFO: Ignoring response <401 https://api.elsevier.com/content/search/scopus?query=fundus>: HTTP status code is not handled or not allowed
2021-03-16 16:44:52 [scrapy.core.engine] INFO: Closing spider (finished)
2021-03-16 16:44:52 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 642,
 'downloader/request_count': 2,
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 1416,
 'downloader/response_count': 2,
 'downloader/response_status_count/308': 1,
 'downloader/response_status_count/401': 1,
 'elapsed_time_seconds': 

RAN 
FIN:  elsevier_api.csv


In [46]:
import requests, json


# # print( resp )
# # print( dir(resp) )
# # print( resp.headers )
# ##json.dumps(resp.json(), sort_keys=True, indent=4, separators=(',', ': '))
# ## write to file 
# def save_to_file(data, fname):
#     with open(fname, 'a', encoding='utf-8') as f:
#         json.dump(data, f, ensure_ascii=False, indent=4)

        
# pg = 0
# TOTAL_ITEMS = 0
# PAGEZ = 10 
# # while pg is not None:
# # for pg in range(PAGEZ): 
# while pg < PAGEZ:
#     ## get page    
#     resp = requests.get(f"http://api.elsevier.com/content/search/scopus?query=fundus&count=25&start={pg*25}", #af-id(60032114)+OR+af-id(60022265)
#                     headers={'Accept':'application/json',
#                              'X-ELS-APIKey': MY_API_KEY})
#     REZ = resp.headers['X-ELS-Status']
#     if REZ.strip() != 'OK':
#         print( f"*****{REZ}")
#         break
#     pg += 1
#     ## get next page id  and content object 
#     for k, v in resp.json().items():
# #         print(k)
#         for r, v in resp.json()['search-results'].items():
#             if r == 'opensearch:totalResults' and TOTAL_ITEMS == 0:
#                 TOTAL_ITEMS = int( v )
#                 PAGEZ = TOTAL_ITEMS//25
#                 mod = TOTAL_ITEMS%25
#                 PAGEZ += 0 if mod == 0 else 1 
#                 print( "*** TOTAL_ITEMS: ", TOTAL_ITEMS, ", << ", PAGEZ)
# #             if r == 'Page':
# #                 pg = int(v)
# #                 print("On page: ", pg )
# #                 pg = None 
#             if r == 'entry':
# #                 print( r, len(v) )
# #                 i = 0                
#                 ## save to file
#                 save_to_file(v, f"elsevier_api_{PAGEZ}.json")
# #                 for rec in v:
# #                     print( "\n===\n", len(rec), type(rec) )
# #                     for x, y in rec.items():
# #                         print(f"\t{x}: \t{y}")
# #                     if i == 0:
# #                         break
#                 print(f"PG: {pg} dumped to file")
# #             else:
# #                 print( r, v)
    
# #     break 

In [13]:
from pyscopus import Scopus

In [15]:
scopus = Scopus(MY_API_KEY)

#scopus.search("http://api.elsevier.com/content/search/scopus?query=fundus", count=20)

In [49]:
resp = requests.get("http://api.elsevier.com/content/search/scopus?query=fundus&count=25", #af-id(60032114)+OR+af-id(60022265)
                    headers={'Accept':'application/json',
                             'X-RateLimit-Limit':'1',                             
                             'X-ELS-APIKey': MY_API_KEY})


for k, v in resp.headers.items():
    print("{:20}  {}".format(k, v))

DEBUG: Starting new HTTP connection (1): api.elsevier.com:80
DEBUG: http://api.elsevier.com:80 "GET /content/search/scopus?query=fundus&count=25 HTTP/1.1" 308 None
DEBUG: Starting new HTTPS connection (1): api.elsevier.com:443
DEBUG: https://api.elsevier.com:443 "GET /content/search/scopus?query=fundus&count=25 HTTP/1.1" 200 None


Date                  Tue, 16 Mar 2021 15:01:18 GMT
Content-Type          application/json;charset=UTF-8
Transfer-Encoding     chunked
Connection            keep-alive
allow                 GET
Content-Encoding      gzip
Vary                  Accept-Encoding, Origin
X-ELS-APIKey          6f08887e7863302c0431a454c96fb54c
X-ELS-ReqId           4168da3b7d3379d8
X-ELS-ResourceVersion  default
X-ELS-Status          OK
X-ELS-TransId         97eb66061e9acd6d
X-RateLimit-Limit     20000
X-RateLimit-Remaining  19411
X-RateLimit-Reset     1616393076000
CF-Cache-Status       DYNAMIC
cf-request-id         08dd2746a400001b5fb83cd000000001
Expect-CT             max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"
Server                cloudflare
CF-RAY                630edb1ddc701b5f-NBO


In [8]:
def run_search(query, fname):
    def save_to_file(data, fname):
        with open(fname, 'a', encoding='utf-8') as f:
            json.dump( data, f, ensure_ascii=False, indent=4)
            f.write(",\n")

    pg = 0
    TOTAL_ITEMS = 0
    PAGEZ = 10 
    PG_MAX = 25
    # while pg is not None:
    # for pg in range(PAGEZ): 
    while pg < PAGEZ:
        ## 1. get page    
        resp = requests.get(f"http://api.elsevier.com/content/search/scopus?query={query}&count={PG_MAX}&start={pg*PG_MAX}", #af-id(60032114)+OR+af-id(60022265)
                        headers={'Accept':'application/json',
                                 'X-ELS-APIKey': MY_API_KEY})
        REZ = resp.headers['X-ELS-Status']
        if REZ.strip() != 'OK':
            print( f"*****{REZ}")
            break
        pg += 1
        
        ## 2. get next page id  and content object 
        for k, v in resp.json().items(): 
            for r, v in resp.json()['search-results'].items():
                if r == 'opensearch:totalResults' and TOTAL_ITEMS == 0:
                    TOTAL_ITEMS = int( v )
                    PAGEZ = TOTAL_ITEMS//PG_MAX
                    mod = TOTAL_ITEMS%PG_MAX
                    PAGEZ += 0 if mod == 0 else 1 
                    print( "*** TOTAL_ITEMS: ", TOTAL_ITEMS, ", << ", PAGEZ) 
                    
                if r == 'entry':        
                    ## save to file
                    save_to_file(v, f"0ea_{PAGEZ}__{fname}")
                    print(f"PG: {pg} dumped to file")

In [52]:
searchez = [ ("fundus handcrafted deep learning", 'fundus_hc_dl.json'),
           ("medical image preprocessing", "medical_img_preproc.json"),
           ("image fuse handcrafted deep learning features", 'fuse_hc_dl.json'),
           ("fundus", 'fundus.json')]


for q, f in searchez:
    run_search(q, f)


DEBUG: Starting new HTTP connection (1): api.elsevier.com:80
DEBUG: http://api.elsevier.com:80 "GET /content/search/scopus?query=fundus%20handcrafted%20deep%20learning&count=25&start=0 HTTP/1.1" 308 None
DEBUG: Starting new HTTPS connection (1): api.elsevier.com:443
DEBUG: https://api.elsevier.com:443 "GET /content/search/scopus?query=fundus%20handcrafted%20deep%20learning&count=25&start=0 HTTP/1.1" 200 None
DEBUG: Starting new HTTP connection (1): api.elsevier.com:80


*** TOTAL_ITEMS:  106 , <<  5
PG: 1 dumped to file


DEBUG: http://api.elsevier.com:80 "GET /content/search/scopus?query=fundus%20handcrafted%20deep%20learning&count=25&start=25 HTTP/1.1" 308 None
DEBUG: Starting new HTTPS connection (1): api.elsevier.com:443
DEBUG: https://api.elsevier.com:443 "GET /content/search/scopus?query=fundus%20handcrafted%20deep%20learning&count=25&start=25 HTTP/1.1" 200 6476
DEBUG: Starting new HTTP connection (1): api.elsevier.com:80


PG: 2 dumped to file


DEBUG: http://api.elsevier.com:80 "GET /content/search/scopus?query=fundus%20handcrafted%20deep%20learning&count=25&start=50 HTTP/1.1" 308 None
DEBUG: Starting new HTTPS connection (1): api.elsevier.com:443
DEBUG: https://api.elsevier.com:443 "GET /content/search/scopus?query=fundus%20handcrafted%20deep%20learning&count=25&start=50 HTTP/1.1" 200 None
DEBUG: Starting new HTTP connection (1): api.elsevier.com:80


...

PG: 199 dumped to file


DEBUG: http://api.elsevier.com:80 "GET /content/search/scopus?query=fundus&count=25&start=4975 HTTP/1.1" 308 None
DEBUG: Starting new HTTPS connection (1): api.elsevier.com:443
DEBUG: https://api.elsevier.com:443 "GET /content/search/scopus?query=fundus&count=25&start=4975 HTTP/1.1" 200 None
DEBUG: Starting new HTTP connection (1): api.elsevier.com:80


PG: 200 dumped to file


DEBUG: http://api.elsevier.com:80 "GET /content/search/scopus?query=fundus&count=25&start=5000 HTTP/1.1" 308 None
DEBUG: Starting new HTTPS connection (1): api.elsevier.com:443
DEBUG: https://api.elsevier.com:443 "GET /content/search/scopus?query=fundus&count=25&start=5000 HTTP/1.1" 400 111


*****INVALID_INPUT - Exceeds the number of search results


In [53]:
## data = 16/3 18:37
fundus_search_next_start_at = 5000

 # TODO: Search Language of Scopus
    
    http://schema.elsevier.com/dtds/document/bkapi/search/SCOPUSSearchTips.htm 

In [7]:
import pandas as pd
import numpy as np
import json
from datetime import datetime 


def parse_json_file_to_csv(fname): 
    keyz = [ 'prism:url', 
             "dc:identifier", "dc:title", "prism:publicationName",
             "prism:coverDate", "citedby-count", 
            ]
    affilz = "affiliation" ## sub = affilname, affiliation-city, affiliation-country
    first_affilz_country = "country_1"
    first_affilz_org = "institution_1"
    
    has_fundus_in_title = "fundus_in_title"
    pub_year = "pub_year"
    
    abhref = 'abstract'
    
    def parse_elsevier_json_item(item):    
        
        def get_pub_year(pub_date):
            return str( datetime.strptime(pub_date, "%Y-%m-%d").year )
        
        def has_fundus(titl):
            return str( int( 'fundus' in titl.lower() ) )
        
        O_ = []
        ## 1. everything top level 
        for k in keyz:
            O_.append( item[k] ) 
        
        ## 2. parse affiliation details and join org,city,country
        k_affz = ['affilname', 'affiliation-city', 'affiliation-country']
        A_ = []
        I_ = '-'
        C_ = '-'
        ia = 0 
        affitem = item.get("affiliation", None)
        if affitem:
            for aff in affitem:
                if aff:
                    a_ = []
                    for k in k_affz:
                        x = aff.get(k, "-") 
                        a_.append( x if x is not None else "-")
#                     print( a_ )
                    A_.append(", ".join(a_) )         
                    if ia == 0:
                        C_ = a_[-1]  
                        I_ = a_[0]
                        ia = 100 
            O_.append( "++".join(A_) )
        else:
            O_.append( "-" )
        O_.append( I_ )
        O_.append( C_ )
        
        ## 3. preprocs
        O_.append( has_fundus( item.get("dc:title", "") ) )
        O_.append( get_pub_year(item.get("prism:coverDate", "")))
        
        ## 4. parse abstract url 
        abstract = "link" # @ref = "self", "@href" 
        for abl in item[abstract]:
            if abl['@ref'] == 'self':
                O_.append( abl['@href'] ) 
        
        return O_ 
    
        
    def fetch_abstract(ahref):
        pass 
    
    to_csv = []
    with open( f'{fname}.json', 'r') as fd:
        pagez = json.load( fd)
#         print( type(pagez), len(pagez) )
        for page in pagez:
            for item in page:
                to_csv.append( parse_elsevier_json_item(item) )
            
    
    def dump_csv_row(rec):
#         print(type(rec), len(rec), rec)
        fd.write( "\t".join( rec) )
        fd.write("\n")
        
    with open( f"{fname}.csv", 'w') as fd:
        headz = keyz + [affilz, first_affilz_org, first_affilz_country, 
                        has_fundus_in_title, pub_year, 
                        abhref] 
        dump_csv_row(headz)
        for rec in to_csv:
            dump_csv_row(rec)
            
        

In [ ]:
datz = ['0ea_4__fuse_hc_dl', 
        '0ea_5__fundus_hc_dl', 
        '0ea_642__medical_img_preproc', 
        '0ea_3926__fundus'
       ]

# for fname in datz:
#     parse_json_file_to_csv(fname)
#     print("******** FINISHED - ", fname )

In [10]:
#### ======== FIND ME THESE GUYS
from pyscopus import Scopus
import requests, json
#Cells: 7, 8

MY_API_KEY = '6f08887e7863302c0431a454c96fb54c'
headerz_api={'Accept':'application/json', 'X-ELS-APIKey': MY_API_KEY}


scopus = Scopus(MY_API_KEY)
searchez = [ ("unsupervised survey intensity threshold based fundus blood vessels", 'vessels-paper1.json'),]


for q, f in searchez:
    run_search(q, f)

*** TOTAL_ITEMS:  28 , <<  2
PG: 1 dumped to file
PG: 2 dumped to file


In [16]:
parse_json_file_to_csv('0ea_2__vessels-paper1')